In [1]:
from databases import PosicaoDm1,Crm,BDS, Boletador
from funcoes_datas import FuncoesDatas
import pandas as pd
from datetime import datetime,date,timedelta
from dateutil.relativedelta import relativedelta
import re
import numpy as np 
from emailer import Email 
import math
import locale
from pretty_html_table import build_table

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chamando as classesdo DataBases
posicao = PosicaoDm1()
crm= Crm()
fdt = FuncoesDatas()
bds = BDS()
fdt = FuncoesDatas()
hoje = fdt.hoje()
bol = Boletador()

In [3]:
def moeda(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')

In [4]:
#Posicao ultimo_mes
carteira_atual = posicao.posicao_total_depara(hoje)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_atual=carteira_atual.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_atual['GuidProduto']= carteira_atual['GuidProduto'].str.lower()

#Retirar da carteira Caixa e Compromissada
carteira_atual = carteira_atual[(carteira_atual['TipoProduto']!='CAIXA')&(carteira_atual['TipoProduto']!='COMP')]

In [5]:
#POsicao M-2
#Posicao ultimo_mes
carteira_atual = posicao.posicao_total_depara(hoje)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_atual=carteira_atual.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_atual['GuidProduto']= carteira_atual['GuidProduto'].str.lower()

#Retirar da carteira Caixa e Compromissada
carteira_atual = carteira_atual[(carteira_atual['TipoProduto']!='CAIXA')&(carteira_atual['TipoProduto']!='COMP')]

In [6]:
#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Unifica as bases
ativos_comite = pd.concat([cnpj,isin,cetip])

#Retorna apenas os clientes que passaram pelo comite e suas posições
string = carteira_atual['NomeContaCRM'].drop_duplicates().to_list()
substr = ativos_comite['Cliente'].drop_duplicates().to_list()

def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

clientes_comite = Filter(string, substr)


NameError: name 'produtos' is not defined